## Third Movement: 

In [39]:
# Import required libraries
import networkx as nx
import numpy as np
from scipy.special import factorial
from datetime import datetime
import logging
from scipy.stats import geom
import sys
import os

In [40]:
import os
experiment_log_file = 'experiment_4.log'
log_path = os.path.join(os.getcwd(), experiment_log_file)

if os.path.exists(log_path):
    os.remove(log_path)

logger = logging.getLogger()
handler = logging.FileHandler(log_path, mode='w')
handler.setFormatter(logging.Formatter('%(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

logger.info(f'[1] {experiment_log_file}')
logger.info(f'[1] "{datetime.now().strftime("%a %b %d %H:%M:%S %Y")}"')

In [41]:
%run attack_graph_MIR100.ipynb

### Use Third Distribution

In [42]:
attack_rate_list = [0]   
defense_rate_list = [1, 2, 3]  

In [43]:
def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
    # Get hardness values from the graph
    hardness = []
    for i in range(len(route) - 1):
        u, v = route[i], route[i+1]
        edges_dict = graph[u][v]

        if isinstance(edges_dict, float):
            total_prob = edges_dict
        elif hasattr(edges_dict, 'items'):
            total_prob = 0.0
            for edge_key, attr_dict in edges_dict.items():
                if isinstance(attr_dict, float):
                    total_prob += attr_dict
                else:
                    p = float(attr_dict.get('prob', 1.0))
                    total_prob += p
        else:
            p = float(edges_dict.get('prob', 1.0))
            total_prob = p

        hardness.append(total_prob)

    hardness = np.array(hardness)
    hardness = np.nan_to_num(hardness, nan=1.0)

    # Calculate geometric mean for attack rate
    def geometric_mean(x):
        positive_values = x[x > 0]
        return np.exp(np.mean(np.log(positive_values)))

    if attack_rate is None:
        attack_rate = 1 / geometric_mean(hardness)

    # Calculate probability for geometric distribution
    if attack_rate == 0 and defense_rate == 0:
        # When both rates are 0, return uniform distribution like R does
        return np.ones(len(route)) / len(route)
        
    prob = attack_rate / (attack_rate + defense_rate)
    
    # Generate geometric distribution
    x = np.arange(len(route))
    pdf = geom.pmf(x, prob)
    
    # Handle any remaining NaN values by converting to uniform distribution
    if not np.any(np.isfinite(pdf)):
        pdf = np.ones_like(pdf)
    
    # Normalize
    pdf = pdf / np.sum(pdf)
    
    return pdf

In [44]:
# %run ctr-core_simple.ipynb
%run ctr-core_tests.ipynb
main()


++++++++++++++++++++++++++++++++
attack_rate = 0, defense_rate = 1
--- Starting payoff calc for check = 10, path = [0, 1, 5, 15, 'c(12,13,14,16)'] ---

Avatar: 1 is in path.
Route from avatar: [1, 5, 15, 'c(12,13,14,16)']
pdf_d for entire route: [0.25 0.25 0.25 0.25]
Cut point (including check node): 4
Route subset: [1, 5, 15, 'c(12,13,14,16)']
payoffDistr (normalized up to cutPoint): [0.25 0.25 0.25 0.25]
L distribution for this avatar (BEFORE weighting by theta):
  Node 1: 0.25
  Node 15: 0.25
  Node 5: 0.25
  Node c(12,13,14,16): 0.25
Avatar: 15 is in path.
Route from avatar: [15, 'c(12,13,14,16)']
pdf_d for entire route: [0.5 0.5]
Cut point (including check node): 2
Route subset: [15, 'c(12,13,14,16)']
payoffDistr (normalized up to cutPoint): [0.5 0.5]
L distribution for this avatar (BEFORE weighting by theta):
  Node 15: 0.5
  Node c(12,13,14,16): 0.5
Avatar: 5 is in path.
Route from avatar: [5, 15, 'c(12,13,14,16)']
pdf_d for entire route: [0.33333333 0.33333333 0.33333333]
Cut 

In [45]:
with open(experiment_log_file, 'r') as f:
    print(f.read())

[1] experiment_4.log
[1] "Tue Dec 24 08:51:30 2024"

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  1 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 1.028977e-01
15 3.813853e-01
5 0.000000e+00
6 0.000000e+00
7 1.028977e-01
8 3.099216e-01
9 1.028977e-01

worst case attack strategies per goal:
          1
1 0.0000000
2 0.0000000
3 0.1890962
4 0.0000000
5 0.0000000
6 0.1596267
7 0.1596267
8 0.2458252
9 0.0000000
10 0.2458252
11 0.0000000
[1] 0.062

Defender can keep attacker success below: 0.062
Attacker can guarantee success probability of: 0.062

++++++++++++++++++++++++++++++++

The virtual target nodeID is c(12,13,14,16)

attack rate =  0 , defense rate =  2 

	equilibrium for multiobjective security game (MOSG)

optimal defense strategy:
         prob.
10 0.000000e+00
11 1.028977e-01
15 3.813853e-01
5 0.000000e+00
6 0.000000e+00
7 1.028977e-01
8 3.09

In [46]:
# def random_steps(route, attack_rate=None, defense_rate=None, graph=None):
#     """
#     Calculate probability distribution based on edge difficulties.
    
#     Args:
#         route: List of nodes representing an attack path
#         attack_rate: Not used in this implementation but kept for API consistency
#         defense_rate: Not used in this implementation but kept for API consistency
#         graph: NetworkX graph containing edge probabilities
        
#     Returns:
#         numpy array: Probability distribution for number of steps attacker can take
#     """
#     # Get edge probabilities between consecutive nodes in route
#     hardness = []
#     for i in range(len(route)-1):
#         # Get edge data between these nodes
#         edges_data = graph.get_edge_data(route[i], route[i+1])
        
#         # Handle both MultiDiGraph and DiGraph edge data structures
#         if isinstance(edges_data, dict):
#             if 'edge_probabilities' in edges_data:
#                 # DiGraph case - direct dictionary
#                 prob = float(edges_data.get('edge_probabilities', 1.0))
#                 hardness.append(prob)
#             else:
#                 # MultiDiGraph case - dictionary of edge dictionaries
#                 probs = []
#                 for key in edges_data:
#                     if isinstance(edges_data[key], dict):
#                         prob = edges_data[key].get('edge_probabilities', 1.0)
#                         probs.append(float(prob))
#                     else:
#                         probs.append(float(edges_data.get('edge_probabilities', 1.0)))
#                 hardness.append(max(probs) if probs else 1.0)
#         else:
#             # No edge data found
#             hardness.append(1.0)
    
#     # Convert to numpy array and handle missing values
#     hardness = np.array(hardness)
#     hardness[np.isnan(hardness)] = 1.0  # Missing probabilities treated as certainty
    
#     # Calculate PDF according to the paper's formula
#     # P(n steps) = (1-p_{n+1}) * product(p_1 to p_n)
#     # where p_i is probability of success at step i
    
#     # Calculate cumulative products
#     cumprod = np.concatenate(([1.0], np.cumprod(hardness)))
    
#     # Calculate (1-p) terms
#     one_minus_h = np.concatenate((1 - hardness, [1.0]))
    
#     # Final PDF calculation
#     pdf = one_minus_h * cumprod
    
#     # Normalize to ensure it sums to 1
#     pdf = pdf / np.sum(pdf)
    
#     return pdf